# Use
This notebook will be used to tests the functions of the python files

In [336]:
import numpy as np
import warnings
%load_ext autoreload
%autoreload 2
from hungarian import preprocess, alternate, hungarian

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test of `preprocess(C)`

In [337]:
n=4
C = np.random.random(size=(n,n))
n,U,V,row,x = preprocess(C)
print(C-U[:,np.newaxis]-V,"\n",x)

[[0.59216455 0.37469491 0.         0.55524684]
 [0.04597711 0.60900007 0.         0.54268023]
 [0.         0.         0.91171088 0.        ]
 [0.33012166 0.68357131 0.         0.0381224 ]] 
 [[False False  True False]
 [False False False False]
 [ True False False False]
 [False False False False]]


In [338]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,5,7],
              [1,6,6,9],
              [3,6,2,2]])
n,U,V,row,x = preprocess(C)
print(C-U[:,np.newaxis] - V, row, U, V,"\n", x)

[[0 0 1 2]
 [0 4 3 5]
 [0 3 5 8]
 [1 2 0 0]] [0 None 3 None] [7 2 1 2] [0 2 0 0] 
 [[ True False False False]
 [False False False False]
 [False False False False]
 [False False  True False]]


seem ok
- The zero entries match the True entries
- Internal assertions satisfied
- Same results as exemple 4.3
## Test of `alternate(C,U,V,row,k)`

In [339]:
n=4
C = np.random.randint(10,size=(n,n))
n,U,V,row,x = preprocess(C)
k=n-1
if np.any(row==k):
     print("Invalid candidate k, try again")
else:
    sink,pred,SU,LV = alternate(C,U,V,row,k)
    print(row)
    print(sink,pred,SU,LV)
    print(C-U[:,np.newaxis]-V)

Invalid candidate k, try again


In [340]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,6,7],
              [1,6,6,9],
              [3,6,2,2]])
n,U,V,row,x = preprocess(C)
k=1
sink,pred,SU,LV = alternate(C,U,V,row,k)
print(sink,pred)

1 [1 0 None None]


## Tests of `hungarian(C)`

In [341]:
# Exemple 4.3
n=4
C = np.array([[7,9,8,9],
              [2,8,6,7],
              [1,6,6,9],
              [3,6,2,2]])
row,x,phi,U,V = hungarian(C)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)
[2 0 1 3] [1 2 0 3] [8 6 5 2] [-4  1  0  0]
[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]]


In [342]:
# random integer
n=10
C = np.random.randint(0,10,(n,n))
row,x,phi,U,V = hungarian(C)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)
[8 1 4 3 0 2 7 6 5 9] [4 1 5 3 2 8 7 6 0 9] [2 1 2 0 1 2 2 1 1 4] [-1  2 -1  0 -1 -2 -1  1 -1  1]
[[0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]]


In [343]:
# random gaussian
n=4
C = np.random.random(size=(n,n))
row,x,phi,U,V = hungarian(C)
print(C)
print(C-U[:,np.newaxis]-V)
print(row,phi,U,V)
print(1*x)

hungarian succed (feasibility and complementary slackness holds)
[[0.39474292 0.32925682 0.62591257 0.01171872]
 [0.44045576 0.59690404 0.83515698 0.20061209]
 [0.45735615 0.22446194 0.86045797 0.47653075]
 [0.03311328 0.85275745 0.36774661 0.67588245]]
[[0.14318053 0.07769443 0.03971686 0.        ]
 [0.         0.15644828 0.06006789 0.        ]
 [0.23289421 0.         0.3013627  0.49191248]
 [0.         0.81964418 0.         0.88261284]]
[1 2 3 0] [3 0 1 2] [0.25156239 0.44045576 0.22446194 0.03311328] [ 0.          0.          0.33463333 -0.23984367]
[[0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]


In [344]:
# random gaussian
n=100
d=10
a=np.random.randn(d,n)
b=np.random.randn(d,n)
C = np.sum((a[:,:,np.newaxis]-b[:,np.newaxis,:])**2,axis=0)
#print(C)
row,x,phi,U,V = hungarian(C)
#print(C-U[:,np.newaxis]-V)
#print(row,phi,U,V)
#print(1*x)

hungarian succed (feasibility and complementary slackness holds)


## Questions/remarks
- We must test the **scalar equality** $u_i+v_j=c_{ij}$ multiple times, for now done with a tolerance of proximity but it's not very absolute and strong. How resolve? Solutions:
    - Use integers by **rescaling**.(the more we rescale the slower it becomes)
    - During the dual updating, **save the index** where the constrain is becoming active (by saving the argmin, see how to do it)
    
    -> it is indeed true with tolerance but wont affect the convergence of the algorithm neither the final output significantly

- implementation of **sets** by **vectors**. Better by python sets?

    -> we'll see, not so bad 

- describe the functionment of algorithm? a sketch of background?

    -> yes, sketch the idea, describe the steps, invariants, stopping situation.

- with floats, there is sometimes a small transgression of the dual feasibility, has been dealed by adding a tolerance `tol=1e-5`.